In [1]:
import pandas as pd
import numpy as np
import seaborn

import statsmodels.api as sm

In [2]:
# Missing symbols [u'CMCSA', u'KFT', u'SLE', u'WAG']
# All NaN values [BNI, DELL, HNZ, NIX]
symbols = ["AAPL", "MMM", "ABT", "AA", "ALL", "MO", "AMZN", "AEP", "AXP", "AMGN",
           "T", "AVP", "BHI", "BAC", "BK", "BAX", "BA", "BMY", "BNI", "CPB",
           "COF", "CAT", "CVX", "CSCO", "C", "KO", "CL", "COP", "COST",
           "CVS", "DELL", "DVN", "DOW", "DD", "EMC", "ETR", "EXC", "XOM", "FDX",
           "F", "FCX", "GD", "GE", "GILD", "GS", "GOOG", "HAL", "HPQ", "HNZ",
           "HD", "HON", "INTC", "IBM", "JNJ", "JPM", "LMT", "LOW", "MA",
           "MCD", "MDT", "MRK", "MET", "MSFT", "MON", "MS", "NOV", "NWSA", "NKE",
           "NSC", "NYX", "OXY", "ORCL", "PEP", "PFE", "PM", "PG", "QCOM", "RTN",
           "RF", "SLB", "SO", "S", "TGT", "TXN","TWX", "UNH", "UPS",
           "UTX", "USB", "VZ", "WMT", "DIS", "WFC", "WMB", "WY", "XRX"]

data = get_pricing(
    symbols,
    fields='close_price',
    start_date='2014-01-01',
    end_date = '2015-08-30',
    frequency='minute'
)

In [43]:
# Backfill nan's 
data.fillna(method='bfill', inplace=True)
# Estimate the daily CV from the minute data
df = (data.resample('D', how='std') / data.resample('D', how='mean')).dropna(axis=0, how='all')

In [61]:
# Estimate the squared r for each pair fo symbols
R_results = np.zeros(len(symbols) - 1)
for i in range(len(symbols) - 1):
    results = sm.OLS(df.ix[:, 0], df.ix[:, i+1]).fit()
    R_results[i] = results.rsquared

R_df = pd.DataFrame(R_results, columns=["squared_r"] , index=df.columns[1:])

In [62]:
R_df.describe()

,squared_r
count,91.000000
mean,0.683246
std,0.047680
min,0.458411
25%,0.658470
50%,0.690842
75%,0.715632
max,0.753681


In [68]:
# Peek the 20 with the squared R respect to AAPL
selected_stocks = R_df.sort("squared_r", ascending=False)[:20]
selected_stocks

,squared_r
Equity(12160 [COF]),0.753681
Equity(20088 [GS]),0.752018
Equity(8151 [WFC]),0.746235
Equity(25090 [HON]),0.743091
Equity(25006 [JPM]),0.742670
Equity(17080 [MS]),0.739850
Equity(25010 [USB]),0.738982
Equity(21418 [MET]),0.737268
Equity(1582 [CL]),0.733595
Equity(1900 [CSCO]),0.732865


In [67]:
for ticket in selected_stocks.index:
    print '"{0}",'.format(ticket.symbol),

"COF", "GS", "WFC", "HON", "JPM", "MS", "USB", "MET", "CL", "CSCO", "HPQ", "BAC", "HD", "IBM", "C", "PG", "PEP", "GE", "CVS", "ABT",
